In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df_original = pd.read_csv("/kaggle/input/seattle-house-sales-prices/house_sales.csv")

In [ ]:
df = df_original.copy()
df.head()

In [ ]:
df.shape

In [ ]:
df.drop(["id", "date"], inplace=True, axis=1)

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
for column in df.columns:
    if df[column].isnull().any():
        print("The {} has {} null values".format(column, df[column].isnull().sum()))

In [ ]:
df.describe()

In [ ]:
plt.boxplot(df['bedrooms'], widths=0.5)

In [ ]:
df['bedrooms'].value_counts()

In [ ]:
df.dtypes

In [ ]:
# removing rows having bedrooms 33 and 11 sonce they are outliers
df.drop(df[df['bedrooms']==33].index, inplace=True, axis=0)
df.drop(df[df['bedrooms']==11].index, inplace=True, axis=0)

In [ ]:
df['bedrooms'].value_counts()

In [ ]:
df.describe()

In [ ]:
df.corr()

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(df.corr(), xticklabels=df.columns, yticklabels=df.columns, linewidths=0, annot=df.corr())

In [ ]:
df1 = df.copy()

In [ ]:
cols = list(df1.columns)
price_col = cols.pop(cols.index('price'))
price_col
df1 = df1[cols+['price']]

In [ ]:
df1.head()

In [ ]:
corr_matrix = df1.corr()
type(corr_matrix)
sort = corr_matrix.sort_values('price',ascending=False)

In [ ]:
sort[['price']]

In [ ]:
plt.scatter(df1['sqft_living'], df1['price'])

In [ ]:
plt.scatter(df1['long'], df1['price'])

In [ ]:
sns.regplot(df1['long'], df1['price'])

In [ ]:
sns.regplot(df1['sqft_living'], df1['price'])

In [ ]:
from numpy import cov
covariance = cov(df1['long'],df1['price'])
covariance

In [ ]:
df['waterfront'].value_counts()

In [ ]:
df['view'].value_counts()

In [ ]:
df['condition'].value_counts()

In [ ]:
#ANOVA
grouped_test2=df1[['waterfront', 'price']].groupby(['waterfront'])
grouped_test2.head()

In [ ]:
from scipy.stats import f_oneway
f_val, p_val = f_oneway(grouped_test2.get_group(0)['price'], grouped_test2.get_group(1)['price'])  
 
print( "ANOVA results: F=", f_val, ", P =", p_val)  

In [ ]:
from scipy.stats import pearsonr
corr, p = pearsonr(df1['long'], df1['price'])
print("correlation: ", corr)
print("p-value: ", p)

In [ ]:
pearson_coef, p_value = pearsonr(df1['yr_renovated'], df1['price'])
print("The Pearson Correlation Coefficient: ", pearson_coef, "\nP-value of P: ", p_value)

In [ ]:
pearson_coef, p_value = pearsonr(df1['long'], df1['price'])
print("The Pearson Correlation Coefficient: ", pearson_coef, "\nP-value of P: ", p_value)

In [ ]:
features1 = df1.columns.to_list()[:-1]
features1

In [ ]:
pearson_coef = []
p_values = []
for column in features1:
    coef, p_val = pearsonr(df1[column], df1['price'])
    pearson_coef.append(coef)
    p_values.append(p_val)

In [ ]:
dict1 = {"Features" : features1, "Pearson Coeffs" : pearson_coef, "P-values" : p_values }

In [ ]:
pd.DataFrame(dict1).sort_values(by="Pearson Coeffs", ascending=False,)

In [ ]:
#taking part of columns and validating the accuracy
features_1 = df1[['sqft_living','grade','sqft_above','sqft_living15','bathrooms','view','sqft_basement','bedrooms','lat','waterfront']]

In [ ]:
features_1.shape

In [ ]:
label = df['price']
label.shape

In [ ]:
featuresall = df1.iloc[:,:-2]
featuresall.head()

In [ ]:
df[featuresall.columns.to_list()[0]].describe()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
bruteScores = []
for i in range(1,30):
    x_train,  x_test, y_train, y_test = train_test_split(featuresall, label, test_size = 0.3, random_state=i)
    
    LinReg_model = LinearRegression().fit(x_train, y_train)
    
    test_score = LinReg_model.score(x_test,y_test)
    train_score = LinReg_model.score(x_train,y_train)
    
    if test_score > train_score:
         bruteScores.append((train_score, test_score, i))
             
print(pd.DataFrame(bruteScores, columns=["train score", 'test score', 'random state']))

In [ ]:
#correlation b/w features
for i in featuresall.columns.to_list():
    for j in featuresall.columns.to_list():
        correlation = df[i].corr(df[j])
        if correlation > 0.95 and correlation != 1:
            print ("Corelated columns: {} {}".format(i,j))

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

In [ ]:
poly = PolynomialFeatures(degree=2)
poly_features = poly.fit_transform(featuresall)

In [ ]:
poly_features.shape

In [ ]:
featuresall.shape

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
bruteScores = []
for i in range(1,30):
    x_train,  x_test, y_train, y_test = train_test_split(poly_features, label, test_size = 0.3, random_state=i)
    
    LinReg_model = LinearRegression().fit(x_train, y_train)
    
    test_score = LinReg_model.score(x_test,y_test)
    train_score = LinReg_model.score(x_train,y_train)
    
    if test_score > train_score:
         bruteScores.append((train_score, test_score, i))
             
print(pd.DataFrame(bruteScores, columns=["train score", 'test score', 'random state']))

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
scaled_features = sc.fit_transform(featuresall)

In [ ]:
principalComponents = PCA(n_components=10)
principalComponents.fit(scaled_features,label)

In [ ]:
PCA_Var_ratio = principalComponents.explained_variance_ratio_.tolist()

In [ ]:
sum(PCA_Var_ratio)

In [ ]:
PCA_features = principalComponents.transform(featuresall)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
bruteScores = []
for i in range(1,30):
    x_train,  x_test, y_train, y_test = train_test_split(PCA_features, label, test_size = 0.3, random_state=i)
    
    LinReg_model = LinearRegression().fit(x_train, y_train)
    
    test_score = LinReg_model.score(x_test,y_test)
    train_score = LinReg_model.score(x_train,y_train)
    
    if test_score > train_score:
         bruteScores.append((train_score, test_score, i))
             
print(pd.DataFrame(bruteScores, columns=["train score", 'test score', 'random state']))

#Cross Validation
from sklearn.model_selection import KFold, cross_val_score # import KFold

kfold = KFold(n_splits=20) # Define the split - into 2 folds 
kfold.get_n_splits(poly_features) # returns the number of splitting iterations in the cross-validator
print(kfold) 

for train_index, test_index in kfold.split(poly_features):
    print('train: %s, test: %s' % (train, test))
    X_train, X_test, y_train, y_test = poly_features[train_index], poly_features[test_index], label[train_index], label[test_index]
    print(np.isnan(X_train).sum())
    y_train.replace(to_replace = np.argwhere(np.isnan(y_train), value=np.mean(y_train))   
    print(np.isnan(y_train).sum())
    print(np.isnan(X_test).sum())
    print(np.isnan(y_test).sum())
    LinearRegression().fit(X_train, y_train)
    scores.append(LinearRegression.score(X_test, y_test))

In [ ]:
#cross validaion
from sklearn.model_selection import cross_val_score
cross_val = cross_val_score(LinearRegression(), poly_features, label, cv=5)

In [ ]:
print(np.mean(cross_val))

In [ ]:
#Pipelining 
from sklearn.pipeline import Pipeline
Poly_Input=[('scale',StandardScaler()), ('polynomial', PolynomialFeatures(degree=2,include_bias=False)), ('model',LinearRegression())]

In [ ]:
polypipe = Pipeline(Poly_Input)
polypipe

In [ ]:
polypipe.fit(featuresall, label)

In [ ]:
y_polypipe = polypipe.predict(featuresall)

In [ ]:
y_polypipe[0:3]

In [ ]:
polypipe.score(featuresall, label)

In [ ]:
Scale_Input=[('scale',StandardScaler()), ('model',LinearRegression())]

In [ ]:
scalepipe = Pipeline(Scale_Input)

In [ ]:
scalepipe.fit(featuresall, label)

In [ ]:
y_scalepipe = scalepipe.predict(featuresall)

In [ ]:
y_scalepipe[0:5]

In [ ]:
scalepipe.score(featuresall, label)

### Decision tree regressor

In [ ]:
#Decision tree regressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
x_train,  x_test, y_train, y_test = train_test_split(featuresall, label, test_size = 0.3, random_state=20)

DTR_model = DecisionTreeRegressor().fit(x_train, y_train)

test_score = DTR_model.score(x_test,y_test)
train_score = DTR_model.score(x_train,y_train)

print("Test Score: {}\nTrain Score: {}".format(test_score,train_score))

In [ ]:
#Decision tree regressor
from sklearn.tree import DecisionTreeRegressor
for i in range(1,30):
    x_train,  x_test, y_train, y_test = train_test_split(featuresall, label, test_size = 0.3, random_state=i)
    
    DTR_model = DecisionTreeRegressor().fit(x_train, y_train)

    test_score = DTR_model.score(x_test,y_test)
    train_score = DTR_model.score(x_train,y_train)
    
    if test_score > train_score:
        print("Test Score: {}\tTrain Score: {}\tRandom State: {}".format(test_score,train_score,i))

### Ridge 

In [ ]:
from sklearn.linear_model import Ridge
x_train,  x_test, y_train, y_test = train_test_split(featuresall, label, test_size = 0.3, random_state=20)

Ridge_model = DecisionTreeRegressor().fit(x_train, y_train)

test_score = Ridge_model.score(x_test,y_test)
train_score = Ridge_model.score(x_train,y_train)

print("Test Score: {}\nTrain Score: {}".format(test_score,train_score))

In [ ]:
from sklearn.linear_model import Ridge
x_train,  x_test, y_train, y_test = train_test_split(poly_features, label, test_size = 0.3, random_state=20)

Ridge_model = DecisionTreeRegressor().fit(x_train, y_train)

test_score = Ridge_model.score(x_test,y_test)
train_score = Ridge_model.score(x_train,y_train)

print("Test Score: {}\nTrain Score: {}".format(test_score,train_score))

# Evaluation

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
#Selecting the best model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
x_train,  x_test, y_train, y_test = train_test_split(poly_features, label, test_size = 0.3, random_state=1)
    
LinReg_model = LinearRegression().fit(x_train, y_train)
    
test_score = LinReg_model.score(x_test,y_test)
train_score = LinReg_model.score(x_train,y_train)
    
if test_score > train_score:             
    print('Train Score: {}\nTest Score: {}\nRandom State: {}'.format(train_score, test_score, i))

In [ ]:
LR_prediction = LinReg_model.predict(x_test)

In [ ]:
mse = mean_squared_error(y_test, LR_prediction)
print('The mean square error of price and predicted value is: ', mse)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r_squared = r2_score(y_test, LR_prediction)
print('The R-square value is: ', r_squared)

In [ ]:
LinReg_model.score(x_test, y_test)

# Pipeline Evaluation

In [ ]:
#Pipelining 
from sklearn.pipeline import Pipeline
Poly_Input=[('scale',StandardScaler()), ('polynomial', PolynomialFeatures(degree=2,include_bias=False)), ('model',LinearRegression())]

In [ ]:
polypipe = Pipeline(Poly_Input)
polypipe

In [ ]:
x_train,  x_test, y_train, y_test = train_test_split(featuresall, label, test_size = 0.3, random_state=1)

In [ ]:
polypipe.fit(x_train, y_train)

In [ ]:
y_polypipe = polypipe.predict(x_test)

In [ ]:
y_polypipe[0:4]

In [ ]:
r_squared = r2_score(y_test, y_polypipe)
print('The R-square value is: ', r_squared)